# Jason's Honours Project
## Take 5

In [82]:
# Libraries
import numpy as np
from scipy.constants import mu_0
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageDraw, ImageColor
import matplotlib.pyplot as plt
from scipy.optimize import minimize, NonlinearConstraint, Bounds, brute

In [65]:
# Relative constants
# Things that aren't easily changed
BR_CORE = 1.2
SIGMA_WIRE = 2.3e6
CP_WIRE = 296
RHO_WIRE = 6440
V_MAX = 12
L_TUBE_MAX = 4.5
L_PLUNGER = 0.005
THICKNESS_SHELL = 0.0025
DISTANCE_RG2 = 0.001
REL_PERM_STEEL = 2000
REL_PERM_NEO = 1.05
REL_PERM_AIR = 1

moe = 1e-8

dDrive = 9 # 9mm
fMotor = 9 # 9N
freq = 1 # 1Hz

| x | Name | Units | Description |
| - |------|-------|-------------|
| 0 | rWireOut | mm | Outer radius of silicone tube |
| 1 | thickWireWall | mm | Wall thickness of silicone tube |
| 2 | lCore | mm | Length of motor core |
| 3 | rCore | mm | Radius of motor core |
| 4 | lConnector | mm | How much magnetic connector at the end of the magnet |
| 5 | layers | #NA# | Number of layers of wire |
| 6 | Q | ml/s | Flow rate of liquid metal |

In [83]:
def motorMain(x,rho,sigma,cp,br,dDrive,fMotor,freq,layers):
    
    l = constraintsFun(x) # Check constraints
    
    if not(np.all(np.array(ub)>=np.array(l))) or not(np.all(np.array(lb)<=np.array(l))):
        print("Not valid!")
        return 1e6 # return some really large number
    
    print("motorMain: " + str(x))
    
    rWireOut = x[0] / 1000
    thickWireWall = x[1] / 1000
    lCore = x[2] / 1000
    rCore = x[3] / 1000
    lConnector = x[4] / 1000
    Q = x[5] / 1e6
    dDrive = dDrive / 1000
    
    rWireIn = rWireOut - thickWireWall # Wire inner radius
    
    rShellIn = rCore+rWireOut*layers # Shell inner radius
    
    deltaHeat = rho*Q*cp*deltaT # Heat power difference of liquid metal start and end of motor
    
    rTotal = lWireTotal/np.pi/rWireIn**2/sigma # Total wire resistance
    
    pMech = 2*dDrive*fMotor*freq # Mechanical power output
    
    rg = np.log(rShellIn/rCore)/(2*np.pi*lConnector) # gap reluctance
    
    rmTtl = rg # Total reluctance - approx. gap reluctance.
    
    emmf = br*lCore # electromagneticmotive force
    
    flux = emmf / rmTtl # Magnetic flux through circuit
    
    for range(1,layer+1):
        
    
    B = flux / area_action
    
    I = deltaHeat/(rTotal**2 - B*lWireInField)
    
    power = I * np.sqrt(lWireTotal/(np.pi*rho*rWireIn**2))
    
    print("Power: "+ str(power))
                         
    return power

In [84]:
MAX_LM_VOL = 15 # 15ml Max volume of liquid metal in circuit
MAX_SHELL_DIM = 100 # 10cm Max length of shell
MAX_TEMP_CHANGE = 60 # 60 deg.C Max change in temperature of liquid metal through motor
MIN_TUBE_THICK = 0.5 #0.5mm Min thickness of wire tube

lb = []
ub = []
# 3. Maximum volume of liquid metal in circuit ($ A_{wire} l_{wireTotal} $)
lb.append(0)
ub.append(MAX_LM_VOL)

# 4. Maximum shell dimensions ($ r_{shellIn} $)
lb.append(0)
ub.append(MAX_SHELL_DIM)

# 5. Maximum core length ($ l_{core} $)
lb.append(0)
ub.append(MAX_SHELL_DIM)

# 6. Drive distance doesn't exceed motor dimensions ($ d_{drive}  < \frac{l_{wireTotal} - l_{wireInField}}{2\pi r_{shellIn}} $)
ub.append(dDrive)
lb.append(0)
# This needs to be less than dDrive

#7. Inner motor parts don't exceed outer motor parts ($ r_{shellIn} >  r_{plunger}$, $ r_{shellIn} > r_{core}$, $ r_{plunger} >= r_{core}$)
# All three need to be larger than 0
lb.append(0) #plunger_dimension_check
ub.append(MAX_SHELL_DIM)

lb.append(0) #core_dimension_check
ub.append(MAX_SHELL_DIM)

lb.append(0) #plunger_core_dimension_check
ub.append(MAX_SHELL_DIM)

#8. Maximum temperature change ($ \Delta T $)
# Needs to be smaller than value
lb.append(0)
ub.append(MAX_TEMP_CHANGE)

#9. Total length of wire is larger than wire in field ($ l_{wireTotal} > l_{wireInField} $)
# Needs to be larger than 0
lb.append(0)
ub.append(1000)

#10. Minimum wire wall thickness ($ r_{wireOut} - r_{wireIn} > 0.5 $)
# Needs to be larger than value
lb.append(MIN_TUBE_THICK)
ub.append(1000)

#11. Length of total wire fills a whole layer ($ l_{wireTotal} = 2\pi l_{core}(layers\cdot r_{core}+r_{wireOut}\frac{(1+layers)layers}{2}$)
# Needs to be 0
lb.append(0)
ub.append(0)

#12. Layers is an integer
# if integer, =1, else =0
# needs to be 1
lb.append(1)
ub.append(1)

In [77]:
def constraintsFun(x):
    print("constraintsFun: " + str(x))
    
    lWireTotal = x[0]
    lWireInField = x[1]
    rWireIn = x[2] / 1000
    rWireOut = x[3] / 1000
    lCore = x[4] / 1000
    rCore = x[5] / 1000
    lConnector = x[6] / 1000
    lPlunger = x[7] / 1000
    rPlunger = x[8] / 1000
    layers = x[9]
    Q = x[10] / 1e6
    deltaT = x[11]
    
    rShellIn = rCore+rWireOut*layers # Shell inner radius
    
    l = []
    # 3. Maximum volume of liquid metal in circuit ($ A_{wire} l_{wireTotal} $)
    aWire = rWireIn**2*np.pi
    l.append(aWire*lWireTotal)
    
    # 4. Maximum shell dimensions ($ r_{shellIn} $)
    l.append(rShellIn)
    
    # 5. Maximum core length ($ l_{core} $)
    l.append(lCore)
    
    # 6. Drive distance doesn't exceed motor dimensions ($ d_{drive}  < \frac{l_{wireTotal} - l_{wireInField}}{2\pi r_{shellIn}} $)
    drive_distance_dimension_check = (lWireTotal-lWireInField)/(2*np.pi*rShellIn)
    print(drive_distance_dimension_check)
    l.append(drive_distance_dimension_check)
    # This needs to be less than dDrive
    
    #7. Inner motor parts don't exceed outer motor parts ($ r_{shellIn} >  r_{plunger}$, $ r_{shellIn} > r_{core}$, $ r_{plunger} >= r_{core}$)
    plunger_dimension_check = rShellIn-rPlunger
    core_dimension_check = rShellIn-rCore
    plunger_core_dimension_check = rPlunger-rCore
    # All three need to be larger than 0
    
    l.append(plunger_dimension_check)
    l.append(core_dimension_check)
    l.append(plunger_core_dimension_check)
    
    #8. Maximum temperature change ($ \Delta T $)
    # Needs to be larger than value
    l.append(deltaT)

    #9. Total length of wire is larger than wire in field ($ l_{wireTotal} > l_{wireInField} $)
    # Needs to be larger than 0
    wire_length_check = lWireTotal - lWireInField
    l.append(wire_length_check)
    
    #10. Minimum wire wall thickness ($ r_{wireOut} - r_{wireIn} > 0.5 $)
    # Needs to be larger than value
    wall_thickness_check = rWireOut - rWireIn
    l.append(wall_thickness_check)
    
    #11. Length of total wire fills a whole layer ($ l_{wireTotal} = 2\pi l_{core}(layers\cdot r_{core}+r_{wireOut}\frac{(1+layers)layers}{2}$)
    # Needs to be 0
    whole_layer_check = lWireTotal - 2*np.pi*lCore*layers*(rCore+rWireOut*(1+layers)/2)
    l.append(whole_layer_check)
    
    #12. Layers is an integer
    # if integer, =1, else =0
    # needs to be 1
    if np.abs(int(layers) - layers) > moe:
        l.append(0)
    else:
        l.append(1)
        
    return l

$r_{shellIn} = r_{core}+r_{wireOut}\cdot layers$ Shell inner radius

$\Delta heat = \rho c_p Q \Delta T$ Heat power difference of liquid metal start and end of motor

$R_{total} = (\frac{l_{wireTotal}}{\sigma \pi {r_{wireIn}}^2})^2$

$P_{mechanical} = 2d_{drive}F_{motor}f$

$rg_1 = \frac{ln(\frac{r_{shellIn}}{r_{core}})}{2\pi l_{connector}}$

$rg_2 = \frac{ln(\frac{r_{shellIn}}{r_{plunger}})}{2\pi l_{plunger}}$

$rm_{ttl} = rg_1 + rg_2$

$emmf = B_r * l_{core}$

$\phi = \frac{emmf}{rm_{ttl}}$

$A_{action} = 2\pi l_{connector}(r_{core}+2layers\cdot r_{wireOut}-r_{wireOut})$ 

$B = \frac{\phi}{A_{action}}$

$ I=\frac{\Delta heat}{{R_{total}}^2 - B l_{wireInField}}$

Optimise for minimum power ($I\sqrt{\frac{l_{wireTotal}}{\pi r_{wireIn}^2 \rho}}$):
1. Given force ($ F_{motor} $), drive distance ($ d_{drive} $) and frequency ($f$) 9N - 9mm - 1Hz
2. Given magnet strength ($ B_r $)
3. Maximum volume of liquid metal in circuit ($ A_{wire} l_{wireTotal} $)
4. Maximum shell dimensions ($ r_{shellIn} $)
5. Maximum core length ($ l_{core} $)
6. Drive distance doesn't exceed motor dimensions ($ d_{drive}  < \frac{l_{wireTotal} - l_{wireInField}}{2\pi r_{shellIn}} $)
7. Inner motor parts don't exceed outer motor parts ($ r_{shellIn} >  r_{plunger}$, $ r_{shellIn} > r_{core}$, $ r_{plunger} >= r_{core}$)
8. Maximum temperature change ($ \Delta T $)
9. Total length of wire is larger than wire in field ($ l_{wireTotal} > l_{wireInField} $)
10. Minimum wire wall thickness ($ r_{wireOut} - r_{wireIn} > 0.5 $)
11. Length of total wire fills a whole layer ($ l_{wireTotal} = 2\pi l_{core}(layers\cdot r_{core}+r_{wireOut}\frac{(1+layers)layers}{2}$)
12. Layers is an integer

Notes:
- Q is given to Simran as 0.5 ml/s

In [85]:
startingx = [5, 1, 0.75, 1.25, 10, 5, 10, 5, 10, 3, 0.5, 50]

l = constraintsFun(startingx)
assert(len(ub)==len(lb) and len(ub)==len(l))

print(np.array(ub)>=np.array(l))
print(np.array(lb)<=np.array(l))

constraintsFun: [5, 1, 0.75, 1.25, 10, 5, 10, 5, 10, 3, 0.5, 50]
72.75654541343786
[ True  True  True False  True  True  True  True  True  True False  True]
[ True  True  True  True False  True  True  True  True False  True  True]


In [81]:
constraints = NonlinearConstraint(constraintsFun,lb,ub,keep_feasible=False)
bounds = Bounds(lb=[1,0,0,0.25,0.75,1,1,1,1,1,0,0],ub=[15,15,5,10,100,100,50,50,100,10,10,60])

brute(motorMain, startingx, args=(RHO_WIRE, SIGMA_WIRE, CP_WIRE, BR_CORE, dDrive, fMotor, freq), Ns=1000, tol=moe, constraints=constraints, bounds=bounds)



constraintsFun: [ 5.    1.    0.75  1.25 10.    5.   10.    5.   10.    3.    0.5  50.  ]
72.75654541343786
constraintsFun: [ 5.00000007  1.          0.75        1.25       10.          5.
 10.          5.         10.          3.          0.5        50.        ]
72.75654676863412
constraintsFun: [ 5.          1.00000001  0.75        1.25       10.          5.
 10.          5.         10.          3.          0.5        50.        ]
72.75654514239861
constraintsFun: [ 5.          1.          0.75000001  1.25       10.          5.
 10.          5.         10.          3.          0.5        50.        ]
72.75654541343786
constraintsFun: [ 5.          1.          0.75        1.25000002 10.          5.
 10.          5.         10.          3.          0.5        50.        ]
72.75654494879916
constraintsFun: [ 5.          1.          0.75        1.25       10.00000015  5.
 10.          5.         10.          3.          0.5        50.        ]
72.75654541343786
constraintsFun: [ 5.       

 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.02997845622218
constraintsFun: [ 4.95935112  0.45931849  0.5202828   5.17092886  9.18507083  2.83727308
 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.029978585463915
constraintsFun: [ 4.9593511   0.45931851  0.5202828   5.17092886  9.18507083  2.83727308
 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.029978326980455
constraintsFun: [ 4.9593511   0.45931849  0.52028282  5.17092886  9.18507083  2.83727308
 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.02997845622218
constraintsFun: [ 4.9593511   0.45931849  0.5202828   5.17092888  9.18507083  2.83727308
 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.029978361139335
constraintsFun: [ 4.9593511   0.45931849  0.5202828   5.17092886  9.18507084  2.83727308
 11.36093082  5.18427447  9.79013746  3.          2.92906862 50.14667913]
39.02997845622218


/Users/Guan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Guan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Guan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


     fun: -3194.9962919212085
     jac: array([-4.73015417e+03,  3.91987700e+04,  1.05481933e+05, -3.26142090e+03,
        1.91690973e+03,  1.43658109e+03, -1.68423953e+03, -2.95444427e+02,
        2.96766327e+02, -7.63807968e+03, -9.57548645e+02, -6.29124451e+01])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 187
     nit: 12
    njev: 12
  status: 6
 success: False
       x: array([ 4.93731865,  0.35902615,  0.44281072,  6.22546057,  7.34170451,
        2.43599371, 15.36538937,  7.67581651,  9.20239474,  3.        ,
        3.33664116, 50.78481138])

In [75]:
test = np.array([ 4.93731865,  0.35902615,  0.44281072,  6.22546057,  7.34170451,
        2.43599371, 15.36538937,  7.67581651,  9.20239474,  3.        ,
        3.33664116, 50.78481138])

print(np.array(ub)>=np.array(test))
print(np.array(lb)<=np.array(test))

[ True  True  True  True  True  True  True  True  True  True False False]
[ True  True  True  True  True  True  True  True  True  True  True  True]


In [ ]:
# Graphing